In [1]:
import json
import os
from tqdm import tqdm

data_dir = '../data/TIME_SERIES_DAILY'
TICKER_LIST = '../snp500sym.csv'
write_dir = '../data/MACD_data'

if not os.path.exists(write_dir):
    os.makedirs(write_dir)
    print(f'Created directory {write_dir}')

Created directory ../data/MACD_data


In [2]:
def get_ema(data, dates, num_days):
    wt = 2/(num_days + 1)
    init_ema = 0
    for i in range(num_days):
        init_ema += float(data[dates[i]]['4. close'])
    init_ema /= num_days
    ema = {dates[num_days-1]: init_ema}
    for i in range(num_days, len(dates)):
        ema[dates[i]] = (float(data[dates[i]]['4. close']) * wt + ema[dates[i-1]] * (1 - wt))
    return ema

def get_macd(data, dates):
    if len(dates) <= 26:
        return None
    ema_12 = get_ema(data, dates, 12)
    ema_26 = get_ema(data, dates, 26)
    macd = {date: ema_12[date] - ema_26[date] for date in ema_26.keys()}
    macd_normalised = {date: macd[date] / ema_26[date] for date in ema_26.keys()}
    return macd, macd_normalised

In [3]:
f = open(TICKER_LIST, 'r')
for line in tqdm(f.readlines()):
    ticker = line.strip()
    data = json.load(open(os.path.join(data_dir, ticker + '.json'), 'r'))
    try:
        data = data['Time Series (Daily)']
    except:
        continue
    dates = list(data.keys())
    dates.reverse()
    mcd = get_macd(data, dates)
    if mcd is None:
        continue
    macd, macd_normalised = mcd
    fw = open(os.path.join(write_dir, ticker + '.csv'), 'w')
    fw.write('Date, MACD, MACD_n\n')
    for date in macd.keys():
        fw.write(f'{date}, {macd[date]}, {macd_n[date]}\n')
    fw.close()
    

  0%|          | 0/505 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../data/TIME_SERIES_DAILY/MMM.json'